# Alarm Clock & Reminder App (Beginner-Friendly Python Project)

Welcome! This notebook will guide you step-by-step to build an Alarm Clock & Reminder App in Python. You can run and test each section interactively in Jupyter Notebook.

**Features:**
- Set multiple alarms/reminders with custom messages
- View, edit, and delete alarms
- Alarms trigger with sound or message
- 12-hour and 24-hour time formats
- Optional: Save alarms to file, snooze, repeat reminders

---

## How to Use This Notebook
1. Run each cell in order.
2. Follow the instructions and menu prompts.
3. Try setting, editing, and deleting alarms.
4. Test the alarm trigger and sound alert.

---

**To run in VS Code:**
- Use the provided `.py` script version (see end of notebook).
- Open terminal, run: `python AlarmClock_ReminderApp.py`

Let's get started!

## 1. Import Required Libraries

We'll use only beginner-friendly Python libraries. For sound alerts, `winsound` works on Windows, and `playsound` is cross-platform. We'll also use `datetime`, `time`, `threading`, and `json` for file persistence.

In [ ]:
# Import required libraries
import datetime
import time
import threading
import json

# For sound alerts
try:
    import winsound  # Windows only
except ImportError:
    winsound = None
try:
    from playsound import playsound  # Cross-platform
except ImportError:
    playsound = None

## 2. Helper Functions for Time Formatting

Let's create functions to handle 12-hour and 24-hour time formats, parse user input, and validate time strings. This helps users set alarms in their preferred format.

In [ ]:
# Helper functions for time formatting

def parse_time_input(time_str, time_format='24'):
    """
    Parse time string to datetime.time object.
    Supports 12-hour (e.g., '08:30 PM') and 24-hour (e.g., '20:30') formats.
    """
    try:
        if time_format == '12':
            return datetime.datetime.strptime(time_str, '%I:%M %p').time()
        else:
            return datetime.datetime.strptime(time_str, '%H:%M').time()
    except ValueError:
        return None

def format_time(dt_time, time_format='24'):
    """
    Format datetime.time object to string in desired format.
    """
    if time_format == '12':
        return dt_time.strftime('%I:%M %p')
    else:
        return dt_time.strftime('%H:%M')

def is_valid_time(time_str, time_format='24'):
    """
    Check if the time string is valid for the given format.
    """
    return parse_time_input(time_str, time_format) is not None

## 3. Alarm Data Structure and Storage

We'll store alarms in a list of dictionaries. Each alarm will have:
- `time`: when to trigger
- `message`: custom message
- `repeat`: once, daily, or interval
- `status`: active/inactive

We'll also set up a default file for saving alarms.

In [ ]:
# Alarm data structure and storage

alarms = []  # List to store alarms
ALARM_FILE = 'alarms.json'  # File for persistence

def create_alarm_dict(time_obj, message, repeat='once', status='active'):
    return {
        'time': time_obj.strftime('%H:%M'),  # Store as 24-hour string
        'message': message,
        'repeat': repeat,  # 'once', 'daily', or interval in minutes
        'status': status,
        'snooze_until': None  # For snooze feature
    }

## 4. Set Alarm Function

This function lets you set a new alarm by entering the time, message, and repeat option. It validates input and adds the alarm to the list.

In [ ]:
# Function to set a new alarm
def set_alarm():
    print('--- Set a New Alarm ---')
    time_format = input('Choose time format (12/24): ').strip()
    if time_format not in ['12', '24']:
        print('Invalid format. Defaulting to 24-hour.')
        time_format = '24'
    time_str = input('Enter alarm time (e.g., 08:30 PM or 20:30): ').strip()
    if not is_valid_time(time_str, time_format):
        print('Invalid time format.')
        return
    time_obj = parse_time_input(time_str, time_format)
    message = input('Enter alarm message: ').strip()
    repeat = input('Repeat? (once/daily/interval in minutes): ').strip()
    if repeat.isdigit():
        repeat = int(repeat)
    alarm = create_alarm_dict(time_obj, message, repeat)
    alarms.append(alarm)
    print('Alarm set successfully!')

## 5. View All Alarms Function

This function displays all scheduled alarms in a formatted table, showing time, message, repeat, and status.

In [ ]:
# Function to view all alarms
def view_alarms():
    print('--- Scheduled Alarms ---')
    if not alarms:
        print('No alarms set.')
        return
    print(f"{'Index':<6}{'Time':<8}{'Message':<20}{'Repeat':<10}{'Status':<10}")
    for idx, alarm in enumerate(alarms):
        print(f"{idx:<6}{alarm['time']:<8}{alarm['message']:<20}{str(alarm['repeat']):<10}{alarm['status']:<10}")

## 6. Edit Alarm Function

This function allows you to select an alarm by its index and modify its time or message.

In [ ]:
# Function to edit an alarm
def edit_alarm():
    view_alarms()
    if not alarms:
        return
    try:
        idx = int(input('Enter index of alarm to edit: '))
        alarm = alarms[idx]
    except (ValueError, IndexError):
        print('Invalid index.')
        return
    new_time = input(f"Enter new time (current: {alarm['time']}) or press Enter to keep: ").strip()
    if new_time:
        if is_valid_time(new_time, '24'):
            alarm['time'] = parse_time_input(new_time, '24').strftime('%H:%M')
        else:
            print('Invalid time format. Keeping old time.')
    new_msg = input(f"Enter new message (current: {alarm['message']}) or press Enter to keep: ").strip()
    if new_msg:
        alarm['message'] = new_msg
    print('Alarm updated!')

## 7. Delete Alarm Function

This function lets you remove an alarm by its index.

In [ ]:
# Function to delete an alarm
def delete_alarm():
    view_alarms()
    if not alarms:
        return
    try:
        idx = int(input('Enter index of alarm to delete: '))
        alarms.pop(idx)
        print('Alarm deleted!')
    except (ValueError, IndexError):
        print('Invalid index.')

## 8. Alarm Checker and Trigger Function

This function continuously checks the current time against scheduled alarms and triggers them when the time matches.

In [ ]:
# Function to check and trigger alarms
def check_alarms():
    now = datetime.datetime.now().strftime('%H:%M')
    for alarm in alarms:
        if alarm['status'] == 'active':
            # Snooze logic
            if alarm.get('snooze_until'):
                if datetime.datetime.now().strftime('%H:%M') == alarm['snooze_until']:
                    trigger_alarm(alarm)
                    alarm['snooze_until'] = None
            elif alarm['time'] == now:
                trigger_alarm(alarm)
                # Handle repeat
                if alarm['repeat'] == 'once':
                    alarm['status'] = 'inactive'
                elif alarm['repeat'] == 'daily':
                    pass  # Keep active
                elif isinstance(alarm['repeat'], int):
                    # Interval repeat
                    next_time = (datetime.datetime.now() + datetime.timedelta(minutes=alarm['repeat'])).time()
                    alarm['time'] = next_time.strftime('%H:%M')

## 9. Sound Alert Implementation

This function plays a sound alert when an alarm triggers. It uses `winsound` for Windows and `playsound` for other platforms.

In [ ]:
# Function to play sound alert
def play_sound():
    print('Playing alarm sound...')
    if winsound:
        winsound.Beep(1000, 1000)  # Frequency, duration
    elif playsound:
        # You can use a .wav file path here
        try:
            playsound('alarm.wav')
        except Exception:
            print('Sound file not found or error playing sound.')
    else:
        print('No sound library available.')

def trigger_alarm(alarm):
    print(f"\n⏰ ALARM! {alarm['message']} ({alarm['time']})")
    play_sound()
    # Snooze option
    snooze = input('Snooze? (y/n): ').strip().lower()
    if snooze == 'y':
        mins = input('Snooze for 5 or 10 minutes? (5/10): ').strip()
        if mins in ['5', '10']:
            next_time = (datetime.datetime.now() + datetime.timedelta(minutes=int(mins))).time()
            alarm['snooze_until'] = next_time.strftime('%H:%M')
            alarm['status'] = 'active'
            print(f"Alarm snoozed until {alarm['snooze_until']}")

## 10. Save and Load Alarms (File Persistence)

These functions save alarms to a JSON file and load them when the program starts, so alarms persist between sessions.

In [ ]:
# Functions to save and load alarms
def save_alarms():
    with open(ALARM_FILE, 'w') as f:
        json.dump(alarms, f)
    print('Alarms saved to file.')

def load_alarms():
    global alarms
    try:
        with open(ALARM_FILE, 'r') as f:
            alarms = json.load(f)
        print('Alarms loaded from file.')
    except (FileNotFoundError, json.JSONDecodeError):
        alarms = []

## 11. Main Menu and User Interface

This section builds the CLI menu system with options to set, view, edit, delete alarms, and exit.

In [ ]:
# Main menu and user interface
def main_menu():
    load_alarms()
    while True:
        print('\n--- Alarm Clock & Reminder App ---')
        print('1. Set Alarm')
        print('2. View Alarms')
        print('3. Edit Alarm')
        print('4. Delete Alarm')
        print('5. Save & Exit')
        choice = input('Choose an option (1-5): ').strip()
        if choice == '1':
            set_alarm()
        elif choice == '2':
            view_alarms()
        elif choice == '3':
            edit_alarm()
        elif choice == '4':
            delete_alarm()
        elif choice == '5':
            save_alarms()
            print('Goodbye!')
            break
        else:
            print('Invalid choice. Try again.')

## 12. Background Alarm Monitor

This section runs a background thread that continuously checks for alarms while allowing user interaction with the menu.

In [ ]:
# Background thread to monitor alarms
def alarm_monitor():
    while True:
        check_alarms()
        time.sleep(30)  # Check every 30 seconds

def start_alarm_monitor():
    t = threading.Thread(target=alarm_monitor, daemon=True)
    t.start()

## 13. Snooze Functionality

Snooze is integrated in the alarm trigger function. When an alarm goes off, you can choose to snooze for 5 or 10 minutes.

## 14. Testing the Complete App

Let's run the alarm monitor and start the main menu. You can set, edit, delete alarms, and test the alarm trigger and snooze features.

In [ ]:
# Run the alarm monitor and main menu
start_alarm_monitor()
main_menu()